# Project 3: Web APIs and Classification: Model Benchmarks

In [1]:
#Imports:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import re
from bs4 import BeautifulSoup as bs
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import random
import time

%matplotlib inline

In [2]:
# Set the graph style
plt.style.use('ggplot')

## Reading the dataframe

In [3]:
final_df = pd.read_csv('./datasets/final_df.csv')
final_df

,text,label
0,driven individual rushing towards dream ever s...,1.0
1,reduce bounce rate webpage,1.0
2,made animated summary lean start eric ries hop...,1.0
3,skate ramp business,1.0
4,help getting textile prototype created,1.0
...,...,...
2161,trying learn various ing strategy came across ...,0.0
2162,pretend know lot finance economics sold positi...,0.0
2163,bill ackman bet market recovery despite covid ...,0.0
2164,news covid vaccine drugmaker pfizer pfe partne...,0.0


## Train test split

Split the model into their train and test set before transforming the text using the count vectorizer

In [4]:
X = final_df['text']
y = final_df['label']

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [14]:
print(X_train.shape)
print(X_test.shape)

(1624,)
(542,)


## Transforming the text using `countvectorizer`

In [15]:
# Import CountVectorizer.
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate the "CountVectorizer" object
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = None) 

In [16]:
X_train = vectorizer.fit_transform(X_train)

In [17]:
X_test = vectorizer.transform(X_test)

In [18]:
# Convert X_train into a DataFrame.

X_train_df = pd.DataFrame(X_train.toarray(),
                          columns=vectorizer.get_feature_names())
X_train_df

,aa,aaa,aapl,aar,aaron,aaxn,aaz,ab,abandon,abbv,...,zm,zoetis,zone,zoo,zookeeper,zoom,zts,zuck,zuckerberg,zweig
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1621,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Convert X_test into a DataFrame.
X_test_df = pd.DataFrame(X_test.toarray(),
                         columns=vectorizer.get_feature_names())

X_test_df

,aa,aaa,aapl,aar,aaron,aaxn,aaz,ab,abandon,abbv,...,zm,zoetis,zone,zoo,zookeeper,zoom,zts,zuck,zuckerberg,zweig
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
538,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
540,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Baseline Accuracy

In [20]:
y_test.value_counts(normalize=True)

1.0    0.538745
0.0    0.461255
Name: label, dtype: float64

The baseline accuracy is required to check if the model performs better than the default model.

The majority class fall in the entrepreneur subreddit which is the class 1.

## Training on the logistics regression model

In [22]:
# Import the logistic regression model
from sklearn.linear_model import LogisticRegression

In [24]:
# Instantiate the Logistic Regression model, setting max_iter to a higher value to prevent convergence warning.
lr = LogisticRegression(solver='newton-cg',max_iter=500)

# Fit the model to the training data
lr.fit(X_train, y_train)

# Evaluate the model on the training set
lr.score(X_train, y_train)

0.9956896551724138

In [26]:
# Evaluate the model on the test set
lr.score(X_test, y_test)

0.9059040590405905

It seems the model is overfitting comparing against the train and test set as the train set has a higher accuracy score compared to the test test.

I can reduce the number of features in the model to reduce the variance which will decrease the overfitting and help improve the accuracy score.

I can also increase regularization strength of the model to reduce the overfitting.

In [41]:
from sklearn.metrics import confusion_matrix

y_pred = lr.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print("True Negative:", tn)
print("False Positive:", fp)
print("True Positive:", tp)
print("False Negative:", fn)

True Negative: 219
False Positive: 31
True Positive: 272
False Negative: 20


In [39]:
confusion_matrix(y_test, y_pred)

array([[219,  31],
       [ 20, 272]], dtype=int64)

In [54]:
specificity = tn / (tn+fp) # How accurately can the model predict for the negative class
sensitivity = tp / (tp+fn) # How accurately can the model predict for the positive class

print('Specficity:', round(specificity,2))
print('Sensitivity:',round(sensitivity,2))

Specficity: 0.88
Sensitivity: 0.93


The Sensitivity is slightly higher compared to the Specificity, which means the model is slightly more likely to accurately predict the positive class compared to the negative class. However, since I'm trying to predict whether the model is able to accurately predict the subreddit where the post belongs to, optimizing for Sensitivity and Specificity would not be a good measure. Accuracy would be a better performance metric.

In [55]:
# AUC ROC Curve

## Training on the Multi-nominal Bayes Model

I'll be using Multi-nominal Bayes as the X column is filled with the integer counts of the terms in each document.

In [ ]:
# Import the Multinominal Naive bayes
from sklearn.naive_bayes import MultinomialNB

## Futher Model evaluation